In [1]:
import os
os.chdir(os.path.abspath(".."))


import librosa
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from src.dataload import load_audio_data, audio_select_mean_vote
from src.emotion_translation import ekman_emotion

In [5]:
df_annotations = pd.read_excel("data/annotations_2.xlsx")
df_training_audios = df_annotations[df_annotations['Type'] == 'Train'][['Audio_Name','Part_Num']].drop_duplicates()

In [9]:
# Main loop for audio data
for index, row in df_training_audios.iterrows():
    
    # Input
    data, time, sr = load_audio_data(df_annotations, part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_input = pd.DataFrame(data = np.stack([time, data]).T, columns = ['Time','Data'])
    
    # Labels
    df_label = audio_select_mean_vote(df_annotations,  part_num = row['Part_Num'], audio_name = row['Audio_Name'])
    df_label['Emotion'] = df_label.apply(lambda row : ekman_emotion(row['Valence'], row['Arousal'], row['Dominance']), axis = 1)
    
    time_index = pd.concat([df_label['Time'],df_input['Time']], ignore_index = True).drop_duplicates().sort_values().reset_index(drop = True)
    df_train = pd.merge(time_index, df_input, how = 'left', on = 'Time')
    df_train = pd.merge(df_train, df, how = 'left', on = 'Time')
    
    df_train = df_train.fillna(method='ffill').fillna(method='bfill')
    break